## Working with Shapely - points, lines, polygons, topology

In [16]:
from shapely.geometry import Polygon, Point, LineString
import math
from geopy.distance import vincenty, great_circle



In [11]:
from osgeo import ogr, osr, gdal

# Enable GDAL/OGR exceptions
gdal.UseExceptions()
# example GDAL error handler function

def gdal_error_handler(err_class, err_num, err_msg):
    errtype = {
            gdal.CE_None:'None',
            gdal.CE_Debug:'Debug',
            gdal.CE_Warning:'Warning',
            gdal.CE_Failure:'Failure',
            gdal.CE_Fatal:'Fatal'
    }
    err_msg = err_msg.replace('\n',' ')
    err_class = errtype.get(err_class, 'None')
    print 'Error Number: %s' % (err_num)
    print 'Error Type: %s' % (err_class)
    print 'Error Message: %s' % (err_msg)

gdal.PushErrorHandler(gdal_error_handler)

0

In [12]:
#gdal is having trouble finding the *.csv files which are in "/Applications/anaconda/share/gdal"
gdal.SetConfigOption('GDAL_DATA','/Applications/anaconda/share/gdal')
print(gdal.GetConfigOption('GDAL_DATA'))

/Applications/anaconda/share/gdal


In [13]:
SEA = (47.449, -122.309) # Seattle Tacoma International Airport (SEA)
DFW = (32.896, -97.037) # Dallas / Fort Worth International Airport (DFW)

In [14]:
sea_pt = Point(SEA[1], SEA[0])
dfw_pt = Point(DFW[1], DFW[0])
line_sea_dfw = LineString([sea_pt,dfw_pt])
line_sea_dfw.length

29.162712373851637

In [20]:
print(vincenty(SEA,DFW)) # slower but more accurate
print(great_circle(SEA,DFW))  # great circle compute distance on the sphere not spheriod

2671.13691103 km
2669.03398568 km


If we want to use shapely to calculate length and area it has to be with projected data.
We know how to project a point. Now let's project lines and polygons.
Then we can calculate metrics

In [ ]:
poly = Polygon(((0, 0), (0, 2), (2, 2), (2, 0)))
one_pt = Point(1,1)
one_pt.within(poly)
sfo_pt = Point(sfo[0],sfo[1])
sfo_pt.within(poly)
poly.area
line = LineString([(0, 0), (1, 1)])
line.length


In [21]:
from geopandas import *

In [52]:
# load the shapefile
all_countries = GeoDataFrame.from_file('data/usa_adm1.shp')

# the original CRS of our shapefile and point data
original_crs = all_countries.crs

# the projected CRS to convert our shapefile and point data into
target_crs = {'datum':'WGS84', 'no_defs':True, 'proj':'aea', 'lat_1':35, 'lat_2':55, 'lat_0':45, 'lon_0':10}


In [53]:
# change the CRS of the shapefile to the specified projected one
all_countries_aea = all_countries.to_crs(crs=target_crs, inplace=False)

In [38]:
#all_countries


In [60]:
print(all_countries_aea.NAME_1[1], all_countries_aea.geometry[1].area/(1000 * 1000))
# area of florida in sq km is 170,300 km2)
print(all_countries_aea.NAME_1[45], all_countries_aea.geometry[45].area/(1000 * 1000))
#all_countries.geometry.area
#print(all_countries_aea.NAME_1)

(u'Florida', 147461.75251295223)
(u'California', 409681.2085009259)


In [63]:
# Let's try a different projection
#+proj=aea +lat_1=34 +lat_2=40.5 +lat_0=0 +lon_0=-120 +x_0=0 +y_0=-4000000 +ellps=GRS80 +datum=NAD83 +units=m +no_defs 
cal_aea_crs = {'datum':'NAD83', 'no_defs':True, 'proj':'aea', 'lat_1':34, 'lat_2':40.5, 'lat_0':0, 'lon_0':-120, 'x_0':0, 'y_0': -4000000}
all_countries_ca_aea = all_countries.to_crs(crs=cal_aea_crs, inplace=False)

In [65]:
#area of ca in sq km is 423,970 km2 - so I think this data is low spatial resolution
print(all_countries_ca_aea.NAME_1[45], all_countries_ca_aea.geometry[45].area/(1000 * 1000))

(u'California', 409681.0083207027)
